## **1. Importing necessary libraries**



In [77]:
import numpy as np
import keras
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import layers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import batch_normalization
from keras.layers.activation import LeakyReLU, ReLU

## **2. Loading the dataset**<br>
To load the dataset from local machine to the Google colab, we need to inport drive and mount the data from drive to colab. 

In [78]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [79]:
import os
os.chdir('/content/drive/MyDrive/Data')

## **3. Preprocess the data**<br>
Preprocessing the data by normalizing the pixel values and applying data augmentation techniques such as flipping, rotating, and zooming. This will help the model generalize better to new data.<br><br>
**Performing train, test and validation split of the data. The data has only 300 images in total so I splitted the training and Validation split as 80% and 20% respectively and using the same dataset for testing. It might overfit the model as we have used the previous dataset but the dataset was very small.**

In [80]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, validation_split=0.2)
train_generator = train_datagen.flow_from_directory('./', target_size=(64, 64), batch_size=32, class_mode='categorical')
validation_generator = train_datagen.flow_from_directory('./', target_size=(64, 64), batch_size=32, class_mode='categorical', subset='validation')
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory('./', target_size=(64, 64), batch_size=32, class_mode='categorical', shuffle=False)

Found 300 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Found 300 images belonging to 3 classes.


## **4. Creating the model<br>**
This is a Convolution Neural Network model for classification. We have several convolution layer with relu as activation function, pooling layers and dense layers. <br>
For pooling layers, we have used maxpooling. It helps to reduce dimentionality. 

In [81]:
model = Sequential()

# Add convolutional and pooling layers
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output of the previous layer to feed into dense layers
model.add(Flatten())

# Add dense layers with dropout
model.add(Dense(units=128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.5))

# Output layer with softmax activation for classification
model.add(Dense(units=3, activation='softmax'))

# Compile the model with categorical crossentropy loss and adam optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 64)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 12544)             0         
                                                                 
 dense_3 (Dense)             (None, 128)              

## **5. Compilation of model**<br>

In [83]:
history = model.fit(train_generator, epochs=10)

Epoch 1/10
10/10 [==============================] - 21s 2s/step - loss: 1.2508 - accuracy: 0.3233
Epoch 2/10
10/10 [==============================] - 19s 2s/step - loss: 1.0459 - accuracy: 0.4167
Epoch 3/10
10/10 [==============================] - 18s 2s/step - loss: 0.8663 - accuracy: 0.6100
Epoch 4/10
10/10 [==============================] - 20s 2s/step - loss: 0.6481 - accuracy: 0.7267
Epoch 5/10
10/10 [==============================] - 18s 2s/step - loss: 0.5839 - accuracy: 0.7600
Epoch 6/10
10/10 [==============================] - 19s 2s/step - loss: 0.4917 - accuracy: 0.8100
Epoch 7/10
10/10 [==============================] - 19s 2s/step - loss: 0.5258 - accuracy: 0.8100
Epoch 8/10
10/10 [==============================] - 18s 2s/step - loss: 0.4974 - accuracy: 0.8167
Epoch 9/10
10/10 [==============================] - 19s 2s/step - loss: 0.4237 - accuracy: 0.8333
Epoch 10/10
10/10 [==============================] - 18s 2s/step - loss: 0.3448 - accuracy: 0.8867


## **6. Evaluating model on test dataset**

In [84]:
loss, accuracy = model.evaluate(test_generator)
print('Validation accuracy:', accuracy)
print('Validation loss:', loss)

10/10 [==============================] - 18s 2s/step - loss: 0.1972 - accuracy: 0.9267
Validation accuracy: 0.9266666769981384
Validation loss: 0.19718053936958313


Saving model for resuing it. 

In [ ]:
model.save('CNN_model.h5')